In [6]:
import requests
import csv
import gtfs_realtime_pb2
import time

PB_URL = "https://otd.delhi.gov.in/api/realtime/VehiclePositions.pb?key=qQdrLdHL9z5X2IEPMv1JNx524TutmrQV"

positions = str

cols = ["entity_id","trip","route","start_time","start_date","vehicle_id","lat","lon","veh_speed","veh_timestamp"]
iteration = 1
while True: # Code will run forever without this while statement
    response = requests.get(PB_URL)
    response.raise_for_status()

    feed_message = gtfs_realtime_pb2.FeedMessage()

    feed_message.ParseFromString(response.content)

    with open('output/dtc-rt/dtc-rt-gtfs-output.csv','a',newline='') as csv_file:
        data = []
        writer = csv.writer(csv_file)

        trips,vehicle,position,timestamp = None,None,None,None

        for entity in feed_message.entity:
            entity_id = entity.id
            trip = entity.vehicle.trip.trip_id
            route = entity.vehicle.trip.route_id
            start_time = entity.vehicle.trip.start_time
            start_date = entity.vehicle.trip.start_date

            vehicle_id = entity.vehicle.vehicle.id

            lat = entity.vehicle.position.latitude
            lon = entity.vehicle.position.longitude
            veh_speed = entity.vehicle.position.speed

            veh_timestamp = entity.vehicle.timestamp
            # print(entity)
            data.append([entity_id,trip,route,start_time,start_date,vehicle_id,lat,lon,veh_speed,veh_timestamp])
        print(f"Iteration No: {iteration}")
        iteration += 1
        writer.writerows(data)
    time.sleep(5)

Iteration No: 1
Iteration No: 2


KeyboardInterrupt: 